In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
customer = pd.read_csv('customer.csv', sep=';')
transaction = pd.read_csv('transaction.csv', sep=';')

In [3]:
transaction = transaction.replace({np.nan: None})
customer = customer.replace({np.nan: None})

In [4]:
transaction['list_price']=transaction['list_price'].str.replace(',','.')
transaction['standard_cost']=transaction['standard_cost'].str.replace(',','.')

In [5]:
import psycopg2 

import psycopg2.extras as extras 
import pandas as pd 


def execute_values(conn, df, table): 
    tuples = [tuple(x) for x in df.to_numpy()] 
    cols = ','.join(list(df.columns)) 
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols) 
    cursor = conn.cursor() 
    try: 
        extras.execute_values(cursor, query, tuples) 
        conn.commit() 
        return True
    except (Exception, psycopg2.DatabaseError) as error: 
        print("Ошибка: %s" % error) 
        conn.rollback() 
        return False
    finally:
        cursor.close() 


conn = psycopg2.connect( 
	database="postgres", user='postgres', password='arsen', host='127.0.0.1', port='5433'
) 

In [7]:
execute_values(conn, customer, 'module3.customer') 

True

In [8]:
execute_values(conn, transaction, 'module3.transaction')

True

# Создание базы данных в PostgreSQL при помощи DBeaver

CREATE SCHEMA module3;<br>
<br>
<br>
CREATE table module3.transaction(<br>
transaction_id int4 PRIMARY KEY,<br>
product_id int4 NOT NULL,<br>
brand varchar(30) NULL,<br>
product_line varchar(30) NULL,<br>
product_class varchar(30) NULL,<br>
product_size varchar(30) null,<br>
customer_id int4 NULL,<br>
transaction_date timestamp NOT NULL,<br>
online_order varchar(30) NULL,<br>
order_status varchar(30) NOT NULL,<br>
list_price float4 NOT NULL,<br>
standard_cost float4 NULL,<br>
FOREIGN KEY(customer_id) REFERENCES module1.customer(customer_id)<br>
);<br>
<br>
CREATE table module3.customer(<br>
customer_id int4 PRIMARY KEY,<br>
first_name varchar(50) NULL,<br>
last_name varchar(50) NULL,<br>
gender varchar(30) NULL,<br>
DOB timestamp NULL,<br>
job_title varchar(50) NULL,<br>
job_industry_category varchar(50) NULL,<br>
wealth_segment varchar(50) NULL,<br>
deceased_indicator varchar(50) NULL,<br>
owns_car varchar(30) NULL,<br>
address varchar(50) NULL,<br>
postcode varchar(30) NULL,<br>
state varchar(30) NULL,<br>
country varchar(30) NULL,<br>
property_valuation int4 NULL<br>
);<br>

# Запросы к данным

### Вывести распределение (количество) клиентов по сферам деятельности, отсортировав результат по убыванию количества. — (1 балл)
select job_industry_category, count(*) from module3.customer <br>
where job_industry_category is not null <br>
group by job_industry_category order by count(*) desc;

### Найти сумму транзакций за каждый месяц по сферам деятельности, отсортировав по месяцам и по сфере деятельности. — (1 балл)
select c.job_industry_category, extract(month from transaction_date) as month, sum(t.list_price) from module3.transaction t <br>
inner join  module3.customer c on t.customer_id = c.customer_id and c.job_industry_category is not null <br>
group by c.job_industry_category, extract(month from transaction_date) <br>
order by extract(month from transaction_date), c.job_industry_category;

### Вывести количество онлайн-заказов для всех брендов в рамках подтвержденных заказов клиентов из сферы IT. — (1 балл)
select t.brand, count(*) from module3.transaction t <br>
inner join  module3.customer c on t.customer_id = c.customer_id and c.job_industry_category='IT' <br>
where t.order_status='Approved' and t.online_order='true' and t.brand is not null<br>
group by t.brand;

### Найти по всем клиентам сумму всех транзакций (list_price), максимум, минимум и количество транзакций, отсортировав результат по убыванию 
### суммы транзакций и количества клиентов. Выполните двумя способами: используя только group by и используя только оконные функции. 
### Сравните результат. — (2 балла)
select distinct t.customer_id, c.first_name, c.last_name, sum(t.list_price), min(t.list_price), max(t.list_price), count(t.transaction_id) <br>
from module3.transaction t <br>
inner join  module3.customer c on t.customer_id = c.customer_id <br>
group by t.customer_id, c.first_name, c.last_name<br>
order by sum(t.list_price) desc,count(t.transaction_id) desc;<br>
<br>
select distinct t.customer_id, c.first_name, c.last_name, <br>
sum(t.list_price) over(partition by t.customer_id, c.first_name, c.last_name ) as sum, <br>
min(t.list_price) over(partition by t.customer_id, c.first_name, c.last_name ) as min, <br>
max(t.list_price) over(partition by t.customer_id, c.first_name, c.last_name ) as max, <br>
count(t.transaction_id) over(partition by t.customer_id, c.first_name, c.last_name)  as count<br>
from module3.transaction t <br>
inner join  module3.customer c on t.customer_id = c.customer_id <br>
order by sum desc, count desc;<br>

### Найти имена и фамилии клиентов с минимальной/максимальной суммой транзакций за весь период (сумма транзакций не может быть null). 
### Напишите отдельные запросы для минимальной и максимальной суммы. — (2 балла)
with v1 as (select distinct t.customer_id, c.first_name, c.last_name, sum(t.list_price) as sum <br>
from module3.transaction t <br>
inner join  module3.customer c on t.customer_id = c.customer_id <br>
group by t.customer_id, c.first_name, c.last_name)<br>
select first_name, last_name from v1<br>
where sum = (select min(sum) from v1);<br>
<br>
with v1 as (select distinct t.customer_id, c.first_name, c.last_name, sum(t.list_price) as sum<br>
from module3.transaction t <br>
inner join  module3.customer c on t.customer_id = c.customer_id <br>
group by t.customer_id, c.first_name, c.last_name)<br>
select first_name, last_name from v1<br>
where sum = (select max(sum) from v1);

### Вывести только самые первые транзакции клиентов. Решить с помощью оконных функций. — (1 балл)
select * from ( <br>
select t.*, row_number() over(partition by customer_id order by transaction_date) as num from module3.transaction t) v  <br>
where num=1;

### Вывести имена, фамилии и профессии клиентов, между транзакциями которых был максимальный интервал (интервал вычисляется в днях) — (2 балла).
with v as (select distinct t.customer_id, <br>
max(t.transaction_date) over(partition by customer_id order by transaction_date) as max_d, <br>
min(t.transaction_date) over(partition by customer_id order by transaction_date) as min_d, <br>
max(t.transaction_date) over(partition by customer_id order by transaction_date)- min(t.transaction_date) over(partition by <br>customer_id order by transaction_date) as diff<br>
from module3.transaction t<br>
order by diff desc)<br>
select v.customer_id, c.first_name, c.last_name, c.job_title from v <br>
inner join  module3.customer c on v.customer_id = c.customer_id <br>
where v.diff=(select max(diff) from v);